In [16]:
import rasterio
import numpy as np
import seaborn as sns
import geopandas as gpd 
from rasterstats import zonal_stats
from tqdm.notebook import tqdm

In [32]:
# for given 40 years of data the following ranks should represent the return periods of 5, 10, 15, 20 years
five_return=round(31/5)
ten_return=round(31/10)
fifteen_return=round(31/15)
twenty_return=round(31/20)

def readData(model,scenario):
    maxprecep=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_Climate_Max_{scenario}.tif").read()
    meanprecep=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_Climate_Mean_{scenario}.tif").read()
    stdprecep=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_Climate_STD_{scenario}.tif").read()
    return maxprecep, meanprecep, stdprecep

src=rasterio.open("Data/PrecipitationHistory/CHRIPS_Max_NP_40.tif")
def savefile(array,filename):
    with rasterio.Env():
        profile = src.profile
        profile.update(
            dtype=rasterio.float32,
            count=1,
            compress='lzw')
        with rasterio.open(filename, 'w', **profile) as dst:
            dst.write(array, 1)
def calculateRP(models=['bcc-csm1-1','CanESM2','CNRM-CM5','GFDL-ESM2G','MPI-ESM-MR'],scenarios=['rcp45','rcp85'],rps=[5,10,15,20]):

    for model in models:
        for scenario in scenarios:
            maxprecep, meanprecep, stdprecep=readData(model,scenario)
            arg_sorted=np.argsort(maxprecep,axis=0)[::-1]
            maxprecep_sorted=np.take_along_axis(maxprecep, arg_sorted, 0)
            meanprecep_sorted=np.take_along_axis(meanprecep, arg_sorted, 0)
            stdprecep_sorted=np.take_along_axis(stdprecep, arg_sorted, 0)
            for rp in rps:
                return_rank=round(31/rp)
                mx_design=maxprecep_sorted[return_rank]
                me_design=meanprecep_sorted[return_rank]
                st_design=stdprecep_sorted[return_rank]

                savefile(mx_design,f"Data/ClimateProjections/NepalClimateProjection/{model}_mx_{rp}_{scenario}.tif")
                savefile(me_design,f"Data/ClimateProjections/NepalClimateProjection/{model}_me_{rp}_{scenario}.tif")
                savefile(st_design,f"Data/ClimateProjections/NepalClimateProjection/{model}_st_{rp}_{scenario}.tif")
                
calculateRP()

In [33]:
slopeunits=gpd.read_file('Data/SlopeUnits/SlopeUnits_V3.shp')
coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]

/tmp/ipykernel_17006/2822982207.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]
/tmp/ipykernel_17006/2822982207.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coord_list = [(x,y) for x,y in zip(slopeunits['geometry'].centroid.x , slopeunits['geometry'].centroid.y)]


In [34]:

def readDesign(model,scenario,rp):
    mx_design=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_mx_{rp}_{scenario}.tif")
    mx_design_d=[x[0] for x in mx_design.sample(coord_list)]
    me_design=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_me_{rp}_{scenario}.tif")
    me_design_d=[x[0] for x in me_design.sample(coord_list)]
    st_design=rasterio.open(f"Data/ClimateProjections/NepalClimateProjection/{model}_st_{rp}_{scenario}.tif")
    st_design_d=[x[0] for x in st_design.sample(coord_list)]
    return mx_design_d,me_design_d,st_design_d

def extractRp(models=['bcc-csm1-1','CanESM2','CNRM-CM5','GFDL-ESM2G','MPI-ESM-MR'],scenarios=['rcp45','rcp85'],rps=[5,10,15,20]):

    for model in tqdm(models):
        for scenario in scenarios:
            for rp in rps:
                maxprecep, meanprecep, stdprecep=readDesign(model,scenario,rp)
                slopeunits[f'mx_{rp}_{scenario}_{model}']=np.array(maxprecep)*86400
                slopeunits[f'me_{rp}_{scenario}_{model}']=np.array(meanprecep)*86400
                slopeunits[f'st_{rp}_{scenario}_{model}']=np.array(stdprecep)*86400
extractRp()

  0%|          | 0/5 [00:00<?, ?it/s]

/home/dahala/miniconda3/envs/dlashok/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/dahala/miniconda3/envs/dlashok/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/home/dahala/miniconda3/envs/dlashok/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of call

In [46]:
import pandas as pd
len(pd.read_csv('Data/Covars/AllCovars2.csv'))

71464

In [43]:
slopeunits.drop('geometry',axis=1).to_csv('Data/ClimateProjections/NepalClimateProjection/clim_rps.csv')